# Predição de mortalidade em internações por pneumonias bacterianas sensíveis à Atenção Primária no Brasil, 2017-2021

## Análise preditiva

In [1]:
# IMPORTANDO PACOTES 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd  
import chart_studio.plotly as py
import plotly.graph_objs as gp 
import seaborn as sns

from scipy.stats import iqr
from sklearn.preprocessing import StandardScaler

In [2]:
#IMPOTANDO DADOS
sih_pneumo2 = pd.read_csv("sih_pneumo2_1721_trat.csv")
sih_pneumo2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323012 entries, 0 to 323011
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  323012 non-null  int64  
 1   N_AIH       323012 non-null  int64  
 2   IDADE       323012 non-null  float64
 3   SEXO        323012 non-null  int64  
 4   MORTE       323012 non-null  int64  
 5   DIAS_PERM   323012 non-null  int64  
 6   DIAG_PRINC  323012 non-null  object 
 7   DT_INTER    323012 non-null  object 
 8   ANO_CMPT    323012 non-null  int64  
 9   MES_CMPT    323012 non-null  int64  
 10  MARCA_UTI   323012 non-null  object 
 11  UTI_MES_TO  323012 non-null  int64  
 12  ANO_INT     323012 non-null  int64  
 13  UF          323012 non-null  int64  
dtypes: float64(1), int64(10), object(3)
memory usage: 34.5+ MB


In [3]:
#TRATANDO VARIÁVEIS
sih_pneumo2.drop('Unnamed: 0', axis= 1, inplace= True)
sih_pneumo2['N_AIH'] = sih_pneumo2['N_AIH'].astype(str)
sih_pneumo2['SEXO'] = sih_pneumo2['SEXO'].astype(str)
sih_pneumo2['DIAG_PRINC'] = sih_pneumo2['DIAG_PRINC'].astype(str)
sih_pneumo2['UF'] = sih_pneumo2['UF'].astype(str)
sih_pneumo2['IDADE'] = sih_pneumo2['IDADE'].astype("float64")
sih_pneumo2['DT_INTER'] = pd.to_datetime(sih_pneumo2['DT_INTER'])
sih_pneumo2.info(), sih_pneumo2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323012 entries, 0 to 323011
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   N_AIH       323012 non-null  object        
 1   IDADE       323012 non-null  float64       
 2   SEXO        323012 non-null  object        
 3   MORTE       323012 non-null  int64         
 4   DIAS_PERM   323012 non-null  int64         
 5   DIAG_PRINC  323012 non-null  object        
 6   DT_INTER    323012 non-null  datetime64[ns]
 7   ANO_CMPT    323012 non-null  int64         
 8   MES_CMPT    323012 non-null  int64         
 9   MARCA_UTI   323012 non-null  object        
 10  UTI_MES_TO  323012 non-null  int64         
 11  ANO_INT     323012 non-null  int64         
 12  UF          323012 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(6), object(5)
memory usage: 32.0+ MB


(None,
            N_AIH  IDADE SEXO  MORTE  DIAS_PERM  \
 0  1217100049066   74.0    1      1          8   
 1  1217100051343   63.0    1      0          1   
 2  1216100558520   74.0    3      0          8   
 3  1216100558937   32.0    3      1          7   
 4  1217100046118    1.0    3      0          5   
 
                        DIAG_PRINC   DT_INTER  ANO_CMPT  MES_CMPT  \
 0  J159 - Pneumonia bacteriana NE 2017-01-23      2017         1   
 1  J159 - Pneumonia bacteriana NE 2017-01-11      2017         1   
 2  J159 - Pneumonia bacteriana NE 2017-01-17      2017         1   
 3  J159 - Pneumonia bacteriana NE 2017-01-20      2017         1   
 4  J158 - Pneumonia bacteriana NE 2017-01-08      2017         1   
 
       MARCA_UTI  UTI_MES_TO  ANO_INT  UF  
 0  Não usou UTI           0     2017  12  
 1  Não usou UTI           0     2017  12  
 2  Não usou UTI           0     2017  12  
 3  Não usou UTI           0     2017  12  
 4  Não usou UTI           0     2017  12  )